# Linear Regression in Python

This notebook introduces **linear regression**, one of the fundamental techniques in machine learning. We'll explore how to fit regression models, evaluate their performance, and understand the important distinction between in-sample fit and out-of-sample performance.

## How to Use This Notebook

Remember, all our notebooks are designed for **both in-class work and independent study**. Look for these markers:

- 📚 **AT-HOME** - Complete after class for deeper understanding
- 👤 **SOLO** - You should do this yourself first (build fundamentals)
- 🤖 **AI-ASSISTED** - Use the Antigravity AI assistant for this exercise

---

## Part 1: Regression with `sklearn`

It's very easy to run a linear regression model using the `sklearn` library. Start by loading the necessary libraries:

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt  # To visualize data

Now let's simulate some data to work with:

In [ ]:
# Generate some sample data
np.random.seed(0)
N = 500
X = 2 * np.random.rand(N, 1)
y = 4 + 3 * X + np.random.randn(N, 1)

In [ ]:
plt.scatter(X, y)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Simulated Data')
plt.show()

In [ ]:
model = LinearRegression()
fit = model.fit(X, y)

In [ ]:
print(f'intercept = {fit.intercept_[0]}')
print(f'slope = {fit.coef_[0][0]}')

In [ ]:
y_hat = fit.predict(X)
plt.scatter(X, y)
plt.plot(X, y_hat, color='red')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression Fit')
plt.show()

---

## Part 2: Regression using `statsmodels`

While it's useful to see the `sklearn` library in action (given that we will use it A LOT in this module!), the `statsmodels` library is much better for doing statistics in Python (see the [documentation](https://www.statsmodels.org/stable/index.html)). Start by installing `statsmodels` if you haven't done so already:

In [ ]:
!python -m pip install statsmodels

And import the `statsmodels` API:

In [ ]:
import statsmodels.api as sm

To fit an ordinary least squares regression (what we've simple been calling a linear regression), you type:

In [ ]:
# You can add a constant to your model if you want one
X_b = sm.add_constant(X)
# Instantiate the model
model = sm.OLS(y, X_b)
# Fit the model
fit = model.fit()
# Print the model summary
print(fit.summary())

As you can see, `statsmodels` provides useful information on the model's fit, uncertainty for the estimated coefficients, etc., which `sklearn` does not provide. `statsmodels` has many of the methods and models that you would expect from a library devoted to statistical analysis. For instance, say we wanted to fit our OLS with `robust` standard errors:

In [ ]:
model = sm.OLS(y, X_b)
print(model.fit(cov_type='HC3').summary())

Again, see the `statsmodels` [documentation](https://www.statsmodels.org/stable/index.html) for more information on the models that are supported.

---

## Part 3: In-sample Fit versus Out-of-sample Performance

So far our focus has been trying to find a model that **fits** our data, focusing on ways to update our model **parameters** to reduce **loss**. In machine learning speak, this process is referred to as model **training**. However, training is only one half of supervised machine learning; the other half focuses on **testing** our model using held-out data. To do so, we focus on what's typically referred to as **out-of-sample performance**.

Why "out-of-sample"? In short, it is very easy to **overfit** when using complicated models (i.e., models with lots of weights). Let's get a feel for overfitting and out-of-sample performance by fitting increasingly complex models to our simulated `X` and `y` data above. First, let's start by splitting our data into a **training set** and **testing** set using the `train_test_split` function from `sklearn`:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_b, y, test_size=.2, random_state=42)
print(X_b.shape, X_train.shape, X_test.shape) # notice the difference in dimensions

We'll use a small dataset to illustrate overfitting, as this will make it easier to see. (**NOTE**: I am only doing this for pedagogical reasons, it's typically not a good idea to throw away data!) Let's use the first 25 observations to fit our model:

In [ ]:
X_train = X_train[0:25,:]
y_train = y_train[0:25]

Next, let's re-fit our bivariate model using our training set:

In [ ]:
model1 = sm.OLS(y_train, X_train)
# Fit the model
fit1 = model1.fit()
# Print the model summary
print(fit1.summary())

Now we need to create a more "complicated" model. One way to do this is by introducing additional polynomial terms. For instance, let's start by adding a squared term:

In [ ]:
x2 = X_train[:,1] ** 2 # square x
x2 = x2.reshape(-1, 1) # reshape to column vector
X2_train = np.hstack((X_train, x2)) # merge with original X_train
print(X2_train[0:5,:]) # print first 5 rows

In [ ]:
model2 = sm.OLS(y_train, X2_train)
# Fit the model
fit2 = model2.fit()
# Print the model summary
print(fit2.summary())

What if we kept repeating this process? Let's do it 5 times:

In [ ]:
X_poly_train = X_train
for i in range(2, 5):
    x_poly = X_train[:,1] ** i # square x
    x_poly = x_poly.reshape(-1, 1) # reshape to column vector
    X_poly_train = np.hstack((X_poly_train, x_poly)) # merge with original X_train
    model_poly = sm.OLS(y_train, X_poly_train)
    # Fit the model
    fit_poly = model_poly.fit()
    print(f'x^{i} R^2 = {fit_poly.rsquared}')

The in-sample fit (as measured by R-squared) becomes a little better with each additional term. However, how well do these models perform out-of-sample? To examine this question, let's focus on one common metric for out-of-sample performance for regression problems: the **root mean squared error**. Start by importing the `mean_squared_error` function from `sklearn`:

In [ ]:
from sklearn.metrics import mean_squared_error

Let's get a feel for how this works using our bivariate model:

In [ ]:
# Get predictions from "model1" fit above
y_pred = fit1.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

How does this compare to our polynomial model? First, we need to create the polynomial terms for our test data:

In [ ]:
X_poly_test = X_test
for i in range(2, 5):
    x_poly = X_test[:,1] ** i # square x
    x_poly = x_poly.reshape(-1, 1) # reshape to column vector
    X_poly_test = np.hstack((X_poly_test, x_poly)) # merge with original X_train

In [ ]:
# Get predictions from polynomial model
y_pred = fit_poly.predict(X_poly_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

In summary, we've 1) added additional complexity to our model, 2) got a better in-sample fit (as measured by R-squared), but 3) our out-of-sample fit is considerably worse. That's overfitting!

---

## Part 4: Performance Metrics for Regression

We've already seen the popular **RMSE** metric for judging out-of-sample performance for regression. Honestly, there are not too many others to choose from! We'll quickly look at the two other metrics sometimes used to assess out-of-sample fit for regression models: mean absolute error (MAE) and R-squared for held-out data.

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

Each of these metrics are called in the same way using `sklearn`. For `mean_absolute_error`:

In [ ]:
# Get predictions from "model1" fit above
y_pred = fit1.predict(X_test)

# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error:", mae)

And now using the `r2_score` function:

In [ ]:
# Get predictions from "model1" fit above
y_pred = fit1.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print("Out of sample R-squared:", r2)

---

## Main Exercise: Predicting Housing Prices

Now it's time for you to try on your own! Let's build a model to predict housing prices, using the dataset described [here](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques). Start by loading the `housing.csv` dataset:

In [ ]:
import statsmodels.formula.api as sm # this will allow for R-like formulas
import pandas as pd
data = pd.read_csv('data/housing.csv')
data.head()

<details>
<summary><strong>Click to view the full Codebook</strong></summary>
  
  <br>
MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density
	
LotFrontage: Linear feet of street connected to property

LotArea: Lot size in square feet

Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved
       	
Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access
		
LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular
       
LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression
		
Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	
	
LotConfig: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property
	
LandSlope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope
	
Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker
			
Condition1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
BldgType: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
	
HouseStyle: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level
	
OverallQual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
	
OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
		
YearBuilt: Original construction date

YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)

RoofStyle: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed
		
RoofMatl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles
		
Exterior1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
MasVnrType: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone
	
MasVnrArea: Masonry veneer area in square feet

ExterQual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
ExterCond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood
		
BsmtQual: Evaluates the height of the basement

       Ex	Excellent (100+ inches)	
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches
       NA	No Basement
		
BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement
	
BsmtExposure: Refers to walkout or garden level walls

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement
	
BsmtFinType1: Rating of basement finished area

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement
		
BsmtFinSF1: Type 1 finished square feet

BsmtFinType2: Rating of basement finished area (if multiple types)

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace
		
HeatingQC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
CentralAir: Central air conditioning

       N	No
       Y	Yes
		
Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
		
1stFlrSF: First Floor square feet
 
2ndFlrSF: Second floor square feet

LowQualFinSF: Low quality finished square feet (all floors)

GrLivArea: Above grade (ground) living area square feet

BsmtFullBath: Basement full bathrooms

BsmtHalfBath: Basement half bathrooms

FullBath: Full bathrooms above grade

HalfBath: Half baths above grade

Bedroom: Bedrooms above grade (does NOT include basement bedrooms)

Kitchen: Kitchens above grade

KitchenQual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       	
TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only
		
Fireplaces: Number of fireplaces

FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace
		
GarageType: Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage
		
GarageYrBlt: Year garage was built
		
GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage
		
GarageCars: Size of garage in car capacity

GarageArea: Size of garage in square feet

GarageQual: Garage quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
GarageCond: Garage condition

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
PavedDrive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel
		
WoodDeckSF: Wood deck area in square feet

OpenPorchSF: Open porch area in square feet

EnclosedPorch: Enclosed porch area in square feet

3SsnPorch: Three season porch area in square feet

ScreenPorch: Screen porch area in square feet

PoolArea: Pool area in square feet

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
		
Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence
	
MiscFeature: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None
		
MiscVal: $Value of miscellaneous feature

MoSold: Month Sold (MM)

YrSold: Year Sold (YYYY)

SaleType: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
		
SaleCondition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
</details>

We can get basic descriptive statistics to get a "feel" for our dataset:

In [ ]:
data.describe()

We can see that many of the variables are categorical, so we need to encode them prior to analysis. We will use one-hot encoding for this purpose. For example, take the `MSZoning` variable:

In [ ]:
data = pd.concat([data, pd.get_dummies(data['MSZoning'], prefix='MSZoning')], axis=1)
data.head()

You can now use the encoded variables in subsequent analysis, just make sure to leave one dummy variable out to avoid perfect collinearity!

Next, split the data into a training and testing set:

In [ ]:
data_train, data_test, y_train, y_test = train_test_split(data, data.SalePrice, test_size=.3, random_state=42)
data_train.head()

We can fit a model using the "formula" notation similar to `R`:

In [ ]:
fit = sm.ols(formula="SalePrice ~ LotArea", data=data_train).fit()
print(fit.summary())

We can also get out-of-sample predictions for our model using a formula. However, the syntax is a little different:

In [ ]:
y_pred = fit.predict(exog=data_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error:", mae)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print("Out of sample R-squared:", r2)

---

## 👤 SOLO: Build Your Baseline Model

**Your task:** Build a simple regression model to predict housing prices.

**Requirements:**
1. Choose 2-4 features that you think would be good predictors of housing prices
2. Fit a linear regression model using these features
3. Calculate the out-of-sample RMSE, MAE, and R-squared
4. Document your results in the cell below

**Tip:** Look at the columns in the dataset using `data_train.columns` and review `data_train.describe()` to understand the available features.

In [ ]:
# Explore the available columns
print(data_train.columns)
print("\n")
print(data_train.describe())

In [ ]:
# Your baseline model here
# Example format: fit_baseline = sm.ols(formula="SalePrice ~ Feature1 + Feature2", data=data_train).fit()



In [ ]:
# Evaluate your baseline model
# y_pred_baseline = fit_baseline.predict(exog=data_test)
# Calculate RMSE, MAE, and R-squared



**My baseline results:**

- Features used: 
- RMSE: 
- MAE: 
- R-squared: 

---

## 📚 🤖 (Optional) Feature Engineering for Better Predictions

Now let's use AI to improve your model through **feature engineering**. Feature engineering is the process of creating new features or transforming existing ones to improve model performance.

**Your task:**
1. Copy the prompt template below
2. Ask Antigravity AI to help you engineer better features
3. Review the code carefully (answer review questions)
4. Test it thoroughly

**Prompt Template (copy this to Antigravity chat):**
```
I have a housing price dataset with 81 features. I've built a baseline regression model with [YOUR FEATURES] and achieved an RMSE of [YOUR RMSE] and R-squared of [YOUR R2].

Please help me engineer better features to improve my model. Specifically:
1. Suggest 3-5 new features to create (e.g., interaction terms, polynomial features, or derived features)
2. Provide Python code to create these features in both my training and testing datasets
3. Create a new regression model that includes both the original features and the engineered features
4. Ensure the code handles missing values appropriately
5. Add clear comments explaining what each engineered feature represents

The training data is in data_train and testing data is in data_test.
```

In [ ]:
# Paste your AI-assisted feature engineering code here



In [ ]:
# Paste your AI-assisted model fitting code here



In [ ]:
# Evaluate your enhanced model



### 🔍 Code Review Checkpoint

**Before running the code, answer these (write in cell below):**

1. What new features did AI create? Why might they be useful?
2. How does the code handle missing values?
3. Are the same transformations applied to both training AND testing data?
4. Can you spot any potential issues (e.g., data leakage, overfitting risks)?

**Your answers:**

1. 
2. 
3. 
4. 

### Testing (Required!)

Always test AI-generated code:

In [ ]:
# Test 1: Check that engineered features exist in both train and test
print("Training data shape:", data_train.shape)
print("Testing data shape:", data_test.shape)
print("\nNew columns created:", [col for col in data_train.columns if col not in data.columns])

# Test 2: Check for missing values in new features
print("\nMissing values in new features:")
# [Your test code here]

# Test 3: Verify model performance improved
print("\nBaseline RMSE: [YOUR BASELINE]")
print("Enhanced RMSE: [YOUR ENHANCED]")
print("Improvement: [DIFFERENCE]")

print("\n✅ Tests complete!")

### Reflection

**Answer these questions:**

1. Did the feature engineering improve your model? By how much?
2. Which engineered features do you think are most useful? Why?
3. Did you notice any signs of overfitting (e.g., R-squared getting too high)?
4. What would you try next to improve the model further?

**Your reflection:**

1. 
2. 
3. 
4. 

---

## Wrap-Up

### What You Accomplished Today:

- ✅ Learned how to fit linear regression models with `sklearn` and `statsmodels`
- ✅ Understood the critical difference between in-sample fit and out-of-sample performance
- ✅ Explored the danger of overfitting with polynomial features
- ✅ Learned multiple metrics for evaluating regression models (RMSE, MAE, R²)
- ✅ Built and evaluated models on real housing price data
- ✅ (Optional) Used AI to enhance your model through feature engineering
- ✅ (Optional) Practiced responsible AI-assisted coding (review, test, understand)

### Next Steps:

In the next notebook (gradient descent home learning), you'll learn about the optimization algorithm that powers many machine learning models, including the regression models you've worked with today!